In [5]:
import requests
import pprint
import pandas as pd
import numpy as np
import json
from chicken_dinner.pubgapi import PUBG
import os
from dotenv import load_dotenv
from functions.pubgapi import get_tournament_info

load_dotenv()
api_key = os.environ.get("API_KEY")

In [12]:
tournament_info = get_tournament_info(api_key)
tournament_id = tournament_info["id"]

In [13]:
header = {
  "Authorization": api_key,
  "Accept": "application/vnd.api+json"
}

# tournament_index를 지정하여 원하는 위치의 id값 출력
tournament_index = 9
tournament_index_id = tournament_id[tournament_index]

# 해당 tournament_id를 불러오기 위한 link 작성
url = f"https://api.pubg.com/tournaments/{tournament_index_id}"

# requests.get 함수를 이용하여 tournament_list를 json으로 호출
r = requests.get(url, headers=header)
current_tournament = r.json()

# 모든 match_id와 created_at을 dict으로 출력
matchId_dict = {match['attributes']['createdAt']: match['id'] for match in current_tournament['included']}

# matchId_dict를 createdAt로 정렬 후 dataframe으로 변환
cur_tournament_matchId_df = pd.DataFrame(sorted(matchId_dict.items(), key=lambda x: x[0]), columns=['createdAt', 'matchId'])

In [19]:
# 현재 index에서의 Tournament_id, createdAt, matchId 출력
print(f"Tournament_id : {tournament_id[tournament_index]}\n")
print(cur_tournament_matchId_df)

Tournament_id : as-pp1gf

               createdAt                               matchId
0   2022-04-07T09:53:08Z  d6dd879a-45c9-4412-aa97-3b240b7bdd07
1   2022-04-07T10:39:22Z  a5af5c3b-f389-4e8b-9ae3-bb3e960f55bf
2   2022-04-07T11:24:04Z  c584f09d-1c2c-4c19-9e89-77335801d9e4
3   2022-04-07T12:19:19Z  f6ad1de4-7734-4d02-8b62-25593bb6a19c
4   2022-04-07T13:01:39Z  3be7ab99-296c-4f52-b031-0b35159d12b2
5   2022-04-08T09:45:19Z  7a1a4e65-8ea5-4405-a7a5-a4e1c763d3ac
6   2022-04-08T10:29:15Z  1ab02ce2-1e59-434a-8315-52a897b4a388
7   2022-04-08T11:09:43Z  d35745aa-2d4d-4786-8721-cbb2ae4197c7
8   2022-04-08T11:58:06Z  88e56249-7597-4507-9429-7a5ab84cf6e2
9   2022-04-08T12:45:19Z  dbfd4f45-adab-4d29-937f-4ea921cc602c
10  2022-04-09T09:49:44Z  54a4456f-5402-4be2-8e0e-3500f3d6587b
11  2022-04-09T10:34:29Z  7e7ecfa2-46b9-4e5c-b3ef-87d195efb4ec
12  2022-04-09T11:20:25Z  0a5c82f2-ca15-4085-9dc9-ee60ebcd4d9b
13  2022-04-09T12:06:56Z  074a9705-d83d-4afd-a618-d81e67c21472
14  2022-04-09T12:52:06Z  397

In [20]:
# PUBG class를 토너먼트용 class로 custom
PUBG = PUBG(api_key=api_key, shard='pc-tournament', gzip=True)

# match_participant
player_id = []
team_roster_id = []
team_id = []
team_rank = []
match_id = []
participant_stats = []

# 
for i in cur_tournament_matchId_df['matchId']:
  match = PUBG.match(i)
  rosters = match.rosters
  for i in range(len(rosters)):
    roster = rosters[i]
    roster_participant = roster.participants
    for i in range(len(roster_participant)):
      participant = roster_participant[i]
      match_id.append(match.id)
      player_id.append(participant.name)
      team_roster_id.append(roster.id)
      team_rank.append(roster.stats['rank'])
      team_id.append(roster.stats['team_id'])
      stats = participant.stats
      participant_stats.append(stats)

# Dataframe 생성
match_participant = pd.DataFrame({'match_id': match_id, 'player_id': player_id, 'team_roster_id': team_roster_id, 'team_id': team_id, 'team_rank': team_rank})
match_participant_stats = pd.DataFrame(participant_stats).drop(columns='player_id')

# 인덱스 기준으로 join
match_participant_all = pd.merge(match_participant, match_participant_stats, how='inner', left_index=True, right_index=True) 

# Dataframe 확인
match_participant_all

,match_id,player_id,team_roster_id,team_id,team_rank,dbnos,assists,boosts,damage_dealt,death_type,...,revives,ride_distance,road_kills,swim_distance,team_kills,time_survived,vehicle_destroys,walk_distance,weapons_acquired,win_place
0,d6dd879a-45c9-4412-aa97-3b240b7bdd07,emT_Sparrow,0e3f06d2-c375-4481-bd2d-c625f20cb71c,5,15,1,0,2,154.28500,byplayer,...,1,8815.0640,0,0.0,0,1034,0,663.08000,2,15
1,d6dd879a-45c9-4412-aa97-3b240b7bdd07,emT_JeffLocka,0e3f06d2-c375-4481-bd2d-c625f20cb71c,5,15,1,0,2,117.00255,byplayer,...,0,9390.5440,0,0.0,0,1034,0,352.78350,4,15
2,d6dd879a-45c9-4412-aa97-3b240b7bdd07,emT_DAEVA,0e3f06d2-c375-4481-bd2d-c625f20cb71c,5,15,2,0,1,178.85000,byplayer,...,0,8923.3460,0,0.0,0,1034,0,433.33313,3,15
3,d6dd879a-45c9-4412-aa97-3b240b7bdd07,emT_Stella,0e3f06d2-c375-4481-bd2d-c625f20cb71c,5,15,0,0,2,0.00000,byplayer,...,1,9099.9250,0,0.0,0,1030,0,502.09683,4,15
4,d6dd879a-45c9-4412-aa97-3b240b7bdd07,DK_Makne,17305c92-314a-45f9-a56f-091b8bdb62bf,8,10,1,2,1,141.72096,byplayer,...,0,6211.5186,0,0.0,0,1269,0,675.72160,2,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1275,f630c213-1af6-41c6-bc91-99806f0fe8fc,GBL_Heaven,5040425e-b88f-4deb-894c-abd62c8ba60b,6,14,1,0,2,78.00000,byplayer,...,0,4723.8970,0,0.0,0,1082,0,1192.63000,6,14
1276,f630c213-1af6-41c6-bc91-99806f0fe8fc,MG_SeungYong,fab85343-7207-4b03-b2da-79b079984720,7,13,1,1,4,239.78043,byplayer,...,1,3868.5757,0,0.0,0,1155,0,841.88495,5,13
1277,f630c213-1af6-41c6-bc91-99806f0fe8fc,MG_yacha,fab85343-7207-4b03-b2da-79b079984720,7,13,1,0,0,100.00000,byplayer,...,0,3455.9207,0,0.0,0,836,0,735.44336,3,13
1278,f630c213-1af6-41c6-bc91-99806f0fe8fc,MG_Dlaks,fab85343-7207-4b03-b2da-79b079984720,7,13,0,0,1,0.00000,byplayer,...,0,2643.8503,0,0.0,0,407,0,335.52307,4,13


In [21]:
# 불필요한 항목 제거
## DBNO : Down But Not Out
new_table = match_participant_all.drop(['match_id', 'death_type', 'team_roster_id', 'team_id', 'name', 'vehicle_destroys', 'weapons_acquired'], axis='columns')
new_table

,player_id,team_rank,dbnos,assists,boosts,damage_dealt,headshot_kills,heals,kill_place,kill_streaks,kills,longest_kill,revives,ride_distance,road_kills,swim_distance,team_kills,time_survived,walk_distance,win_place
0,emT_Sparrow,15,1,0,2,154.28500,0,5,31,1,1,16.334719,1,8815.0640,0,0.0,0,1034,663.08000,15
1,emT_JeffLocka,15,1,0,2,117.00255,0,3,30,1,1,14.608637,0,9390.5440,0,0.0,0,1034,352.78350,15
2,emT_DAEVA,15,2,0,1,178.85000,1,1,17,1,2,48.988106,0,8923.3460,0,0.0,0,1034,433.33313,15
3,emT_Stella,15,0,0,2,0.00000,0,0,62,0,0,0.000000,1,9099.9250,0,0.0,0,1030,502.09683,15
4,DK_Makne,10,1,2,1,141.72096,0,1,24,1,1,25.774336,0,6211.5186,0,0.0,0,1269,675.72160,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1275,GBL_Heaven,14,1,0,2,78.00000,1,1,26,1,1,54.208694,0,4723.8970,0,0.0,0,1082,1192.63000,14
1276,MG_SeungYong,13,1,1,4,239.78043,0,3,25,1,1,137.503950,1,3868.5757,0,0.0,0,1155,841.88495,13
1277,MG_yacha,13,1,0,0,100.00000,0,0,57,0,0,0.000000,0,3455.9207,0,0.0,0,836,735.44336,13
1278,MG_Dlaks,13,0,0,1,0.00000,0,0,58,0,0,0.000000,0,2643.8503,0,0.0,0,407,335.52307,13


In [6]:
new_table.keys()

Index(['player_id', 'team_rank', 'dbnos', 'assists', 'boosts', 'damage_dealt',
       'headshot_kills', 'heals', 'kill_place', 'kill_streaks', 'kills',
       'longest_kill', 'revives', 'ride_distance', 'road_kills',
       'swim_distance', 'team_kills', 'time_survived', 'walk_distance',
       'win_place'],
      dtype='object')